![Top <](./images/watsonxdata.png "watsonxdata")

# Federation with watsonx.data
Watsonx.data can federate data from other data sources, there are a few out of box connectors and one could create additional connectors using the SDK if need be (This does involve some programming and testing effort) and not a trivial exercise. We will use the existing PostgreSQL instance, add some data, and test the federation capabilities. 

### PostgreSQL Properties
To set up federation, we need to get the credentials for the PostgreSQL database. Use the following command to get the database password.

In [ ]:
postgres = !cat /certs/passwords | grep Postgres
credentials = postgres[0].split()
print(f"Userid   : {credentials[1]}")
print(f"Password : {credentials[2]}")

You will need to use the watsonx.data UI for the next step. Open your browser and connect to the watsonx.data UI. Navigate to the Infrastructure manager by clicking on the icon below the Home symbol.

![Browser](./images/watsonx-icon-infra.png)
  
You should see a panel like the following.

![Browser](./images/watsonx-infrastructure-1.png)
 
On the top right-hand corner, select Add Component.

![Browser](./images/watsonx-add-component.png)
 
Scroll to the list of Data Sources and select Postgres.

![Browser](./images/watsonx-select-postgres.png)

Press Next.

![Browser](./images/watsonx-add-postgres.png)

Since we will associate the database with a catalog, select the "Associate catalog" checkbox.

![Browser](./images/watsonx-add-postgres-catalog.png)
      
Enter the following values:

   * Display name – <code style="color:blue;font-size:medium;">gosales</code>
   * Database name – <code style="color:blue;font-size:medium;">gosales</code>
   * Hostname –<code style="color:blue;font-size:medium;">ibm-lh-postgres</code>
   * Port – <code style="color:blue;font-size:medium;">5432</code>
   * Username – <code style="color:blue;font-size:medium;">admin</code>
   * Password – The value that was extracted in the earlier step
   * Catalog Name – <code style="color:blue;font-size:medium;">pg_gosales</code>
   
Your screen should look like the one below. You can test the connection at this point. When satisfied that the information is correct, hit the Create button.

![Browser](./images/watsonx-adddb-filled.png)

The infrastructure screen should now show the Postgres database.

![Browser](./images/watsonx-infrastructure-2.png)
 
What we are currently missing the connection between the Presto engine and the data in the gosales database. We must connect the gosales database to the Presto engine. Use your mouse to hover over the pg_gosales icon until you see the Associate connection icon:

![Browser](./images/watsonx-associate-icon.png)
 
Click on the association icon. You should see the following confirmation dialog:

![Browser](./images/watsonx-associate-engine.png)

Select the <code style="color:blue;font-size:medium;">presto-01</code> engine and press **Save and restart engine**.
 
Press the Associate button and the screen will update to show the connection.

![Browser](./images/watsonx-infrastructure-3.png)
 

## Connect to watsonx.data and Presto
This notebook will use magic commands (`%sql`) to issue commands against the watsonx.data Presto database. The next command will load the presto magic command into this notebook.

In [ ]:
%run presto.ipynb

Next we connect to the database. **Note:** The connection may fail if the Presto engine has not completely started. If the connection does fail, just re-run the SQL statement after a minute.

In [ ]:
%%sql
   connect
   userid=ibmlhadmin
   password=password
   hostname=watsonxdata
   port=8443
   catalog=tpch
   schema=tiny
   certfile=/certs/lh-ssl-ts.crt

### Presto Federation

First check to make sure that the Presto engine has finished starting. The following command checks what queries are running in the database. If it fails you know that the presto engine is not yet up and running. 

In [ ]:
%sql select * from "system".runtime.queries limit 5

Connect to the `pg_gosales` catalog. Note that we use a schema of `none` since we don't know which one we want to connect to.

In [ ]:
%sql use pg_gosales.none

Show the current schemas. 

In [ ]:
%sql show schemas

Switch to the `gosalesdw` schema.

In [ ]:
%sql use gosalesdw

Show the tables in the `gosalesdw` schema (there are a lot!).

In [ ]:
%sql show tables

Describe the columns in the MRK_RTL_SURVEY_FACT table.

In [ ]:
%sql show columns in mrk_rtl_survey_fact

Show the columns in the TPCH customer table.

In [ ]:
%sql describe tpch.tiny.customer

Alternate way of describing the columns in the customer table.

In [ ]:
%%sql
prepare tiny from select * from tpch.tiny.customer;
describe output tiny;

Retrieve the first 10 rows from the table.

In [ ]:
%sql select * from tpch.tiny.customer limit 10

Join with data between these two tables (Sample TPCH+PostgreSQL)

In [ ]:
%%sql 
select t.c_name
from   mrk_rtl_survey_fact m, 
       tpch.tiny.customer t
where  m.RETAILER_SURVEY_KEY = t.c_custkey
limit 10

#### Credits: IBM 2025, George Baklarz [baklarz@ca.ibm.com]